In [ ]:

from pyspark.sql.functions import col, sum, dayofweek, hour, mean, when, count 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go
from matplotlib import cm
import matplotlib.colors as mcolors
import requests, zipfile

In [ ]:
df = spark.sql("SELECT * FROM Gold_LH_NYC.fact_trip")
dim_weekday = spark.sql("SELECT * FROM Gold_LH_NYC.dim_weekday")
dim_paymenttype = spark.sql("SELECT * FROM Gold_LH_NYC.dim_paymenttype")
dim_location = spark.sql("SELECT * FROM Gold_LH_NYC.dim_location")

In [ ]:
# 曜日と時間を追加
df = df.withColumn("pickup_weekday", dayofweek(col("pickup_datetime")))  # 'pickup_datetime' 列から曜日を抽出し、新しい列 'pickup_weekday' を追加
df = df.withColumn("pickup_hour", hour(col("pickup_datetime")))  # 'pickup_datetime' 列から時間を抽出し、新しい列 'pickup_hour' を追加

# 曜日と時間ごとの乗客数を集計
# 曜日と時間ごとに 'passenger_count' を集計し、 'total_passenger_count' としてエイリアス
# 'pickup_weekday' と 'pickup_hour' でソート

weekday_hourly_passenger_count = df.groupBy("pickup_weekday", "pickup_hour") \
    .agg(sum("passenger_count").alias("total_passenger_count")).orderBy("pickup_weekday", "pickup_hour")
# マッピングテーブルと結合
weekday_hourly_passenger_count = weekday_hourly_passenger_count.join(dim_weekday, on="pickup_weekday")  # 'pickup_weekday' をキーとして 'dim_weekday' テーブルと結合
# 結果の表示
display(weekday_hourly_passenger_count)  # 結果のデータフレームを表示

In [ ]:
# Pandasデータフレームに変換
weekday_hourly_passenger_count_pd = weekday_hourly_passenger_count.toPandas()  # SparkデータフレームをPandasデータフレームに変換

# ピボットテーブルを作成
pivot_table = weekday_hourly_passenger_count_pd.pivot_table(
    index="description",  # 行ラベルとして 'description' を使用
    columns="pickup_hour",  # 列ラベルとして 'pickup_hour' を使用
    values="total_passenger_count",  # 集計値として 'total_passenger_count' を使用
    aggfunc='sum',  # 集計関数として 'sum' を使用
    fill_value=0  # 欠損値を0で埋める
)

# 曜日順に並び替え
pivot_table = pivot_table.reindex(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"])  # 曜日順に並び替え

# ヒートマップを作成
plt.figure(figsize=(12, 8))  # グラフのサイズを設定
sns.heatmap(
    pivot_table,  # ピボットテーブルをヒートマップとしてプロット
    annot=True,  # 各セルに値を表示
    fmt="d",  # 整数形式で値を表示
    cmap="YlGnBu",  # カラーマップを 'YlGnBu' に設定
    linewidths=.5,  # セル間の線の幅を設定
    cbar_kws={'label': 'Total Passenger Count'}  # カラーバーのラベルを設定
)

# グラフのラベルとタイトル
plt.xlabel('Hour of Day', fontsize=14)  # x軸ラベルを設定
plt.ylabel('Day of Week', fontsize=14)  # y軸ラベルを設定
plt.title('Heatmap of Passenger Count by Day of Week and Hour', fontsize=16)  # グラフのタイトルを設定

plt.tight_layout()  # レイアウトを調整して要素間の重なりを防ぐ
plt.show()  # グラフを表示

In [ ]:
# 時間とBoroughを追加
# 'pickup_datetime' 列から時間を抽出し、新しい列 'pickup_hour' を追加
df = df.withColumn("pickup_hour", hour(col("pickup_datetime")))

# 各Boroughの乗客数を集計
# 時間と 'pickup_LocationID' ごとに 'passenger_count' を集計し、 'total_passenger_count' としてエイリアス
borough_passenger_count = df.groupBy("pickup_hour", "pickup_LocationID").agg(sum("passenger_count").alias("total_passenger_count"))

# ディメンジョンテーブルと結合してBorough情報を追加
# 'pickup_LocationID' をキーとして 'dim_location' テーブルと左結合し、 'Borough' 列を追加
borough_passenger_count = borough_passenger_count.join(dim_location, borough_passenger_count["pickup_LocationID"] == dim_location["LocationID"], "left") \
    .select("pickup_hour", "Borough", "total_passenger_count")

# 各Boroughごとの乗客数を時間ごとに集計
# 時間とBoroughごとに 'total_passenger_count' を集計し、 'total_passenger_count' としてエイリアス
 # 'pickup_hour' と 'Borough' でソート
borough_hourly_passenger_count = borough_passenger_count.groupBy("pickup_hour", "Borough").agg(sum("total_passenger_count").alias("total_passenger_count")).orderBy("pickup_hour", "Borough")

# 結果の表示
display(borough_hourly_passenger_count)

In [ ]:
# Pandasデータフレームに変換
borough_hourly_passenger_count_pd = borough_hourly_passenger_count.toPandas()  # SparkデータフレームをPandasデータフレームに変換

# ピボットテーブルを作成
pivot_table = borough_hourly_passenger_count_pd.pivot(
    index="pickup_hour",  # 行ラベルとして 'pickup_hour' を使用
    columns="Borough",  # 列ラベルとして 'Borough' を使用
    values="total_passenger_count"  # 集計値として 'total_passenger_count' を使用
).fillna(0)  # 欠損値を0で埋める

# プロットの作成
plt.figure(figsize=(12, 8))  # プロットのサイズを設定
pivot_table.plot(
    kind='area',  # 面グラフとしてプロット
    stacked=True,  # スタック形式に設定
    figsize=(12, 8),  # プロットのサイズを再設定
    cmap='tab20'  # カラーマップを 'tab20' に設定
)

# グラフのラベルとタイトル
plt.xlabel('Hour of Day', fontsize=14)  # x軸ラベルを設定
plt.ylabel('Total Passenger Count', fontsize=14)  # y軸ラベルを設定
plt.title('Passenger Count by Hour and Borough', fontsize=16)  # グラフのタイトルを設定
plt.legend(title='Borough', bbox_to_anchor=(1.05, 1), loc='upper left')  # 凡例を設定

plt.tight_layout()  # レイアウトを調整して要素間の重なりを防ぐ
plt.show()  # グラフを表示

In [ ]:
# ピックアップBoroughとドロップオフBoroughの関係を集計
# ピックアップのBorough情報を追加
# ドロップオフのBorough情報を追加
# ピックアップとドロップオフのBoroughごとにカウントを集計
borough_flow = df.join(dim_location, df["pickup_LocationID"] == dim_location["LocationID"], "left") \
    .select(col("pickup_LocationID"), col("dropoff_LocationID"), col("Borough").alias("pickup_Borough")) \
    .join(dim_location, df["dropoff_LocationID"] == dim_location["LocationID"], "left") \
    .select("pickup_Borough", col("Borough").alias("dropoff_Borough")) \
    .groupBy("pickup_Borough", "dropoff_Borough").agg(count("*").alias("count")) \
    .orderBy(col("count").desc())
# 結果の表示
display(borough_flow)  # 結果のデータフレームを表示

In [ ]:
# Pandasデータフレームに変換
borough_flow_pd = borough_flow.toPandas()  # SparkデータフレームをPandasデータフレームに変換

# 出発地点と到着地点のBorough情報を取得
source_boroughs = "Pickup_" + borough_flow_pd['pickup_Borough']  # 'pickup_Borough' 列に 'Pickup_' プレフィックスを追加
target_boroughs = "Dropoff_" + borough_flow_pd['dropoff_Borough']  # 'dropoff_Borough' 列に 'Dropoff_' プレフィックスを追加
all_boroughs = pd.concat([source_boroughs, target_boroughs]).unique()  # 全てのBorough情報を一意に取得
borough_to_index = {borough: idx for idx, borough in enumerate(all_boroughs)}  # Borough情報にインデックスを割り当て

# カラー設定
unique_boroughs = borough_flow_pd['pickup_Borough'].unique()  # 一意なBorough情報を取得
colors = list(mcolors.CSS4_COLORS.values())  # CSS4の全ての色を取得
np.random.shuffle(colors)  # 色をランダムにシャッフル
borough_colors = {borough: color for borough, color in zip(unique_boroughs, colors)}  # Boroughごとに色を割り当て

# ノードの色設定
node_colors = []
for borough in all_boroughs:
    borough_name = borough.split('_')[1]  # プレフィックスを除いたBorough名を取得
    node_colors.append(borough_colors.get(borough_name, 'gray'))  # Borough名に対応する色を取得

# リンクの色設定
link_colors = ['rgba' + str(mcolors.to_rgba(borough_colors[borough], alpha=0.8)) for borough in borough_flow_pd['pickup_Borough']]  # リンクの色を設定

# サンキー図のデータ作成
source_indices = source_boroughs.map(borough_to_index)  # 出発地点のインデックスを取得
target_indices = target_boroughs.map(borough_to_index)  # 到着地点のインデックスを取得
values = borough_flow_pd['count']  # リンクの幅を取得

# サンキー図の作成
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,  # ノードの間隔
        thickness=20,  # ノードの厚さ
        line=dict(color="black", width=0.5),  # ノードの境界線の設定
        label=all_boroughs,  # ノードのラベル
        color=node_colors  # ノードの色
    ),
    link=dict(
        source=source_indices,  # ノードの出発地点
        target=target_indices,  # ノードの到着地点
        value=values,  # リンクの幅
        color=link_colors  # リンクの色
    )
)])

# レイアウトの設定
fig.update_layout(title_text="Taxi Trips from Pickup to Dropoff Boroughs", font_size=12, height=800)  # 図のタイトルとフォントサイズ、高さを設定

# 図の表示
fig.show()  # 図を表示

In [ ]:
# マイルをメートルに変換 (1マイル = 1609.34メートル)
df_distance_meters = df.withColumn("trip_distance_meters", col("trip_distance") * 1609.34)  # 'trip_distance' 列をメートルに変換し、新しい列 'trip_distance_meters' を追加

# 分を時に変換
df_duration_hours = df_distance_meters.withColumn("trip_duration_hours", col("trip_duration") / 60)  # 'trip_duration' 列を分から時に変換し、新しい列 'trip_duration_hours' を追加

# 時速を計算
df_speed_kph = df_duration_hours.withColumn("speed_kph", col("trip_distance_meters") / 1000 / col("trip_duration_hours"))  # 'trip_distance_meters' と 'trip_duration_hours' を使用して時速 (km/h) を計算し、新しい列 'speed_kph' を追加

# 時間帯を分類
df_speed_kph = df_speed_kph.withColumn("time_of_day", 
                                       when((col("pickup_hour") >= 6) & (col("pickup_hour") < 12), "morning")  # 6時から12時を 'morning' に分類
                                       .when((col("pickup_hour") >= 12) & (col("pickup_hour") < 18), "afternoon")  # 12時から18時を 'afternoon' に分類
                                       .when((col("pickup_hour") >= 18) & (col("pickup_hour") < 24), "evening")  # 18時から24時を 'evening' に分類
                                       .otherwise("late night"))  # その他の時間帯を 'late night' に分類

# 結果の確認
df_speed_kph.select("pickup_hour", "speed_kph", "time_of_day").show(5)  # 'pickup_hour', 'speed_kph', 'time_of_day' の最初の5行を表示

# 各時間ごとの平均速度を計算
# 'pickup_hour' と 'time_of_day' ごとに 'speed_kph' の平均を計算し、 'avg_speed_kph' としてエイリアス
hourly_avg_speed = df_speed_kph.groupBy("pickup_hour", "time_of_day").agg(mean("speed_kph").alias("avg_speed_kph")).orderBy("pickup_hour")  # 'pickup_hour' でソート

# 結果の表示
hourly_avg_speed.show()  # 'hourly_avg_speed' データフレームを表示

In [ ]:
# Pandasデータフレームに変換
hourly_avg_speed_pd = hourly_avg_speed.toPandas()  # SparkデータフレームをPandasデータフレームに変換

# カラーマップを設定
color_map = {
    "morning": "orange",
    "afternoon": "blue",
    "evening": "green",
    "late night": "red"
}  # 時間帯ごとの色を設定

# プロットの作成
plt.figure(figsize=(12, 8))  # プロットのサイズを設定

# 時間帯ごとにプロット
for time_of_day, group in hourly_avg_speed_pd.groupby("time_of_day"):
    plt.plot(group['pickup_hour'], group['avg_speed_kph'], marker='o', color=color_map[time_of_day], label=time_of_day)  # 時間帯ごとに 'pickup_hour' と 'avg_speed_kph' をプロット

# グラフのラベルとタイトル
plt.xlabel('Hour of Day', fontsize=14)  # x軸ラベルを設定
plt.ylabel('Average Speed (kph)', fontsize=14)  # y軸ラベルを設定
plt.title('Average Speed by Hour of Day', fontsize=16)  # グラフのタイトルを設定
plt.legend(title='Time of Day')  # 凡例を設定
plt.grid(True)  # グリッドを表示
plt.tight_layout()  # レイアウトを調整して要素間の重なりを防ぐ

# 図の表示
plt.show()  # 図を表示

In [ ]:
# 必要なカラムを選択
df_selected = df.select("trip_distance", "total_amount", "passenger_count")  # 必要なカラム 'trip_distance', 'total_amount', 'passenger_count' を選択

# Pandasデータフレームに変換
df_selected_pd = df_selected.toPandas()  # SparkデータフレームをPandasデータフレームに変換

# 結果の確認
df_selected_pd.head()  # データフレームの最初の5行を表示して確認

In [ ]:
# バブルチャートの作成
plt.figure(figsize=(12, 8))  # プロットのサイズを設定

# バブルチャートのプロット
plt.scatter(
    df_selected_pd['trip_distance'],  # x軸に 'trip_distance' をプロット
    df_selected_pd['total_amount'],  # y軸に 'total_amount' をプロット
    s=df_selected_pd['passenger_count'] * 10,  # バブルのサイズを 'passenger_count' に基づいて設定
    alpha=0.5,  # バブルの透明度を設定
    c='blue',  # バブルの色を設定
    edgecolors='w',  # バブルのエッジカラーを白に設定
    linewidth=0.5  # バブルのエッジの幅を設定
)

# グラフのラベルとタイトル
plt.xlabel('Trip Distance (miles)', fontsize=14)  # x軸ラベルを設定
plt.ylabel('Total Amount ($)', fontsize=14)  # y軸ラベルを設定
plt.title('Bubble Chart: Distance vs Total Amount vs Passenger Count', fontsize=16)  # グラフのタイトルを設定

# グリッドの追加
plt.grid(True)  # グリッドを表示

# 図の表示
plt.tight_layout()  # レイアウトを調整して要素間の重なりを防ぐ
plt.show()  # 図を表示

In [ ]:
# バブルチャートの作成
plt.figure(figsize=(12, 8))  # プロットのサイズを設定

# バブルチャートのプロット
plt.scatter(
    df_selected_pd['trip_distance'],  # x軸に 'trip_distance' をプロット
    df_selected_pd['total_amount'],  # y軸に 'total_amount' をプロット
    s=df_selected_pd['passenger_count'] * 10,  # バブルのサイズを 'passenger_count' に基づいて設定
    alpha=0.5,  # バブルの透明度を設定
    c='blue',  # バブルの色を設定
    edgecolors='w',  # バブルのエッジカラーを白に設定
    linewidth=0.5  # バブルのエッジの幅を設定
)

# 回帰直線の追加
X = df_selected_pd[['trip_distance']]  # 説明変数として 'trip_distance' を設定
y = df_selected_pd['total_amount']  # 目的変数として 'total_amount' を設定
reg = LinearRegression().fit(X, y)  # 線形回帰モデルをフィット
y_pred = reg.predict(X)  # 予測値を計算
plt.plot(df_selected_pd['trip_distance'], y_pred, color='red', linewidth=2, label='Regression Line')  # 回帰直線をプロット

# グラフのラベルとタイトル
plt.xlabel('Trip Distance (miles)', fontsize=14)  # x軸ラベルを設定
plt.ylabel('Total Amount ($)', fontsize=14)  # y軸ラベルを設定
plt.title('Bubble Chart: Distance vs Total Amount vs Passenger Count', fontsize=16)  # グラフのタイトルを設定

# 平均線の追加
mean_total_amount = df_selected_pd['total_amount'].mean()  # 'total_amount' の平均を計算
mean_trip_distance = df_selected_pd['trip_distance'].mean()  # 'trip_distance' の平均を計算
plt.axhline(mean_total_amount, color='green', linestyle='dashed', linewidth=3, label='Mean Total Amount')  # 平均総額の水平線を追加
plt.axvline(mean_trip_distance, color='orange', linestyle='dashed', linewidth=3, label='Mean Trip Distance')  # 平均距離の垂直線を追加

# グリッドの追加
plt.grid(True)  # グリッドを表示

# 凡例の追加
plt.legend()  # 凡例を表示

# 図の表示
plt.tight_layout()  # レイアウトを調整して要素間の重なりを防ぐ
plt.show()  # 図を表示

In [ ]:
!pip install geopandas
!pip install folium
!pip install shapely

In [ ]:
import geopandas as gpd

In [ ]:
nyc_boroughs = gpd.read_file(gpd.datasets.get_path('nybb'))  
# Geopandasのサンプルデータセットからニューヨーク市の区画データを読み込み

# タクシーデータの作成（サンプル）
data = {
    "pickup_LocationID": [1, 2, 3, 4, 5],
    "latitude": [40.7128, 40.7060, 40.7306, 40.7527, 40.7899],
    "longitude": [-74.0060, -74.0086, -73.9352, -73.9772, -73.9730]
}
sampledf = pd.DataFrame(data)  # サンプルのタクシーデータを作成

# GeoDataFrameの作成
gdf = gpd.GeoDataFrame(sampledf, geometry=gpd.points_from_xy(sampledf.longitude, sampledf.latitude))  # タクシーデータをGeoDataFrameに変換

# CRS（座標参照系）の設定
gdf.set_crs(epsg=4326, inplace=True)  # 座標参照系をEPSG:4326に設定

# 地図のプロット
fig, ax = plt.subplots(figsize=(10, 10))  # プロットのサイズを設定

# ニューヨーク市の区画を色分けしてプロット
nyc_boroughs.plot(ax=ax, column='BoroName', legend=True, legend_kwds={'loc': 'upper left'})  # ニューヨーク市の区画データを色分けしてプロット、凡例を左上に配置

In [ ]:
# シェープファイルのURL
url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip"  # シェープファイルが格納されているURL

# ダウンロード先のパス
zip_path = "/lakehouse/default/Files/taxi_zones.zip"  # シェープファイルを保存するローカルのパス

# シェープファイルのダウンロード
response = requests.get(url)  # URLからシェープファイルをダウンロード
with open(zip_path, "wb") as f:  # ダウンロードしたデータをファイルに書き込む
    f.write(response.content)

In [ ]:
# シェープファイルの解凍
with zipfile.ZipFile(zip_path, 'r') as zip_ref:  # ダウンロードしたZIPファイルを開く
    zip_ref.extractall("/lakehouse/default/Files/taxi_zones")  # 指定したディレクトリに解凍


In [ ]:
# 解凍後のシェープファイルのパス
shapefile_path = "/lakehouse/default/Files/taxi_zones/taxi_zones.shp"  # 解凍されたシェープファイルのパス

# タクシーゾーンのシェープファイルを読み込み
taxi_zones = gpd.read_file(shapefile_path)  # Geopandasを使用してシェープファイルを読み込み、GeoDataFrameに変換


In [ ]:
# Pandasデータフレームに変換
dim_location_pd = dim_location.toPandas()  # SparkデータフレームをPandasデータフレームに変換

# シェープファイルとデータを結合
merged = taxi_zones.merge(dim_location_pd, left_on="LocationID", right_on="LocationID")  # タクシーゾーンのシェープファイルとデータをLocationIDで結合

# ニューヨーク市の区画データを読み込み
nyc_boroughs = gpd.read_file(gpd.datasets.get_path('nybb'))  # Geopandasのサンプルデータセットからニューヨーク市の区画データを読み込み

# 地図のプロット
fig, ax = plt.subplots(figsize=(12, 12))  # プロットのサイズを設定

# ニューヨーク市の区画を色分けしてプロット
nyc_boroughs.plot(ax=ax, color='lightgrey', edgecolor='black')  # 区画データをプロット、背景をライトグレー、境界を黒で表示

# タクシーゾーンのプロット
merged.plot(ax=ax, color='white', edgecolor='black', alpha=0.5)  # 結合されたタクシーゾーンデータをプロット、白色で表示し、境界を黒、透明度を0.5に設定

# LocationIDのラベルを追加
for x, y, label in zip(merged.geometry.centroid.x, merged.geometry.centroid.y, merged['LocationID']):  # タクシーゾーンの中心にLocationIDを表示
    ax.text(x, y, label, fontsize=8, ha='right', color='black')  # テキストラベルを追加

# グラフのタイトルとラベル
plt.title('NYC Taxi Zones with Location IDs')  # グラフのタイトルを設定
plt.xlabel('Longitude')  # x軸ラベルを設定
plt.ylabel('Latitude')  # y軸ラベルを設定

plt.show()  # グラフを表示

In [ ]:
# ニューヨーク市の区画データを読み込み
nyc_boroughs = gpd.read_file(gpd.datasets.get_path('nybb'))  # Geopandasのサンプルデータセットからニューヨーク市の区画データを読み込み

# 地図のプロット
fig, ax = plt.subplots(figsize=(12, 12))  # プロットのサイズを設定

# ニューヨーク市の区画を色分けしてプロット
nyc_boroughs.plot(ax=ax, column='BoroName', legend=True, legend_kwds={'loc': 'upper right'}, cmap='Set3')  # 区画データをBoroNameごとに色分けしてプロット、凡例を右上に配置、カラーマップを'Set3'に設定

# タクシーゾーンのプロット
merged.plot(ax=ax, color='white', edgecolor='black', alpha=0.5)  # 結合されたタクシーゾーンデータをプロット、白色で表示し、境界を黒、透明度を0.5に設定

# LocationIDのラベルを追加
for x, y, label in zip(merged.geometry.centroid.x, merged.geometry.centroid.y, merged['LocationID']):  # タクシーゾーンの中心にLocationIDを表示
    ax.text(x, y, label, fontsize=8, ha='right', color='black')  # テキストラベルを追加

# グラフのタイトルとラベル
plt.title('NYC Taxi Zones with Location IDs and Boroughs')  # グラフのタイトルを設定
plt.xlabel('Longitude')  # x軸ラベルを設定
plt.ylabel('Latitude')  # y軸ラベルを設定

plt.show()  # グラフを表示

In [ ]:
# 乗客数の集計
passenger_counts = df.groupBy("pickup_LocationID").agg(sum("passenger_count").alias("total_passenger_count"))  # ピックアップ地点ごとに乗客数を集計

# 乗客数トップ10のピックアップLocationIDを取得
top_10_pickup = passenger_counts.orderBy(col("total_passenger_count").desc()).limit(10)  # 乗客数が多い上位10のピックアップ地点を取得


top_10_pickup_pd = top_10_pickup.toPandas()  # SparkデータフレームをPandasデータフレームに変換
dim_location_pd = dim_location.toPandas()  # SparkデータフレームをPandasデータフレームに変換

# シェープファイルとトップ10データを結合
merged = taxi_zones.merge(top_10_pickup_pd, left_on="LocationID", right_on="pickup_LocationID")  # シェープファイルと乗客数トップ10のデータを結合

 # Geopandasのサンプルデータセットからニューヨーク市の区画データを読み込み
nyc_boroughs = gpd.read_file(gpd.datasets.get_path('nybb')) 

# 地図のプロット
fig, ax = plt.subplots(figsize=(14, 14))  # プロットのサイズを設定

# ニューヨーク市の区画を色分けしてプロット
nyc_boroughs.plot(ax=ax, column='BoroName', legend=True, legend_kwds={'loc': 'upper right'}, cmap='Set3')  # 区画データをBoroNameごとに色分けしてプロット、凡例を右上に配置、カラーマップを'Set3'に設定

# タクシーゾーンのプロット
taxi_zones.plot(ax=ax, color='lightgrey', edgecolor='black', alpha=0.3)  # タクシーゾーンデータをプロット、ライトグレーで表示、境界を黒、透明度を0.3に設定

# バブルのサイズを乗客数に基づいて設定
sizes = merged['total_passenger_count'] / merged['total_passenger_count'].max() * 3000  # バブルのサイズを乗客数に基づいて調整

# バブルチャートのプロット
sc = ax.scatter(merged.geometry.centroid.x, merged.geometry.centroid.y, s=sizes, color='dodgerblue', alpha=0.6, edgecolor='k', linewidth=1.5)  

# LocationIDのラベルを追加
for x, y, label in zip(merged.geometry.centroid.x, merged.geometry.centroid.y, merged['pickup_LocationID']):  # 各ピックアップ地点にLocationIDを表示
    ax.text(x, y, label, fontsize=12, ha='center', color='darkred', weight='bold', bbox=dict(facecolor='white', alpha=0.5, boxstyle='round,pad=0.3'))  # テキストラベルを追加

# グラフのタイトルとラベル
plt.title('Top 10 NYC Taxi Pickup Locations by Passenger Count', fontsize=16, weight='bold')  # グラフのタイトルを設定

plt.show()  # グラフを表示